In [1]:
import pandas as pd
import ipywidgets as widgets
from ipywidgets import Layout, HTML
from IPython.core.interactiveshell import InteractiveShell
from ipyevents import Event
import random
import threading

In [2]:
# Setup global configurations
#InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_rows', 20)

In [3]:
# Load empty board
df = pd.read_csv('board.csv', header=None)

In [4]:
# reset Index
def resetIndex():
    index = []
    for i in range(-4, 21):
        index.append(i)
    df.set_index([pd.Index(index)], inplace=True)
resetIndex()

In [5]:
# Initialize board
output = widgets.Output()
board = widgets.HBox([output], layout=Layout(width='400px', height='600px'))
def refreshBoard(source=None):
    output.clear_output(wait=True)
    with output:
        display(df.iloc[4:24])

In [6]:
# Drop new tetromino
t = {}
def randomTetromino():
    global t
    tetrominos = [{'type':'I', 'rotation':0, 'location':[(1,3), (1,4), (1,5), (1,6)]}, 
                  {'type':'O', 'rotation':0, 'location':[(1,4), (1,5), (2,4), (2,5)]},
                  {'type':'T', 'rotation':0, 'location':[(0,4), (1,3), (1,4), (1,5)]},
                  {'type':'S', 'rotation':0, 'location':[(0,4), (0,5), (1,3), (1,4)]},
                  {'type':'Z', 'rotation':0, 'location':[(0,3), (0,4), (1,4), (1,5)]},
                  {'type':'J', 'rotation':0, 'location':[(0,3), (1,3), (1,4), (1,5)]},
                  {'type':'L', 'rotation':0, 'location':[(0,5), (1,3), (1,4), (1,5)]}]
    t = random.choice(tetrominos)
    for l in t['location']:
        df.iloc[l] = "#"

In [7]:
def move(dir): 
    nt = [] 
    for i in range(0, 4):
        if dir == "down":
            nt.append((t['location'][i][0]+1, t['location'][i][1]))
        elif dir == "left":
            nt.append((t['location'][i][0], t['location'][i][1]-1))
        elif dir == "right":
            nt.append((t['location'][i][0], t['location'][i][1]+1))
            
    blocked = False
    # make sure nothing blocks
    for i in range(0, 4):
        if df.iloc[nt[i]] == "#" and nt[i] not in t['location']:
            blocked = True
            if dir == "down":
                clearRows()
                randomTetromino()
            break
        if nt[i][1] < 0 or nt[i][1] > 10:
            blocked = True
            break
    if not blocked:
        for l in t['location']:
            df.iloc[l] = ' '
        for i in range(0, 4):
            t['location'][i] = nt[i]
        for l in t['location']:
            df.iloc[l] = "#"

In [8]:
def rotate():
    nt = []
    ct = t['location']
    if t['type'] == 'I':
        if t['rotation'] == 0:
            nt.append((ct[0][0]-1, ct[0][1]+2))
            nt.append((ct[1][0], ct[1][1]+1))
            nt.append((ct[2][0]+1, ct[2][1]))
            nt.append((ct[3][0]+2, ct[3][1]-1))
        elif t['rotation'] == 1:
            nt.append((ct[0][0]+2, ct[0][1]+1))
            nt.append((ct[1][0]+1, ct[1][1]))
            nt.append((ct[2][0], ct[2][1]-1))
            nt.append((ct[3][0]-1, ct[3][1]-2))
        elif t['rotation'] == 2:
            nt.append((ct[0][0]+1, ct[0][1]-2))
            nt.append((ct[1][0], ct[1][1]-1))
            nt.append((ct[2][0]-1, ct[2][1]))
            nt.append((ct[3][0]-2, ct[3][1]+1))
        elif t['rotation'] == 3:
            nt.append((ct[0][0]-2, ct[0][1]-1))
            nt.append((ct[1][0]-1, ct[1][1]))
            nt.append((ct[2][0], ct[2][1]+1))
            nt.append((ct[3][0]+1, ct[3][1]+2))
    elif t['type'] == 'T':
        if t['rotation'] == 0:
            nt.append((ct[0][0]+1, ct[0][1]+1))
            nt.append((ct[1][0]-1, ct[1][1]+1))
            nt.append((ct[2][0], ct[2][1]))
            nt.append((ct[3][0]+1, ct[3][1]-1))
        elif t['rotation'] == 1:
            nt.append((ct[0][0]+1, ct[0][1]-1))
            nt.append((ct[1][0]+1, ct[1][1]+1))
            nt.append((ct[2][0], ct[2][1]))
            nt.append((ct[3][0]-1, ct[3][1]-1)) 
        elif t['rotation'] == 2:
            nt.append((ct[0][0]-1, ct[0][1]-1))
            nt.append((ct[1][0]+1, ct[1][1]-1))
            nt.append((ct[2][0], ct[2][1]))
            nt.append((ct[3][0]-1, ct[3][1]+1))
        elif t['rotation'] == 3:
            nt.append((ct[0][0]-1, ct[0][1]+1))
            nt.append((ct[1][0]-1, ct[1][1]-1))
            nt.append((ct[2][0], ct[2][1]))
            nt.append((ct[3][0]+1, ct[3][1]+1))
    elif t['type'] == 'S':
        if t['rotation'] == 0:
            nt.append((ct[0][0]+1, ct[0][1]+1))
            nt.append((ct[1][0]+2, ct[1][1]))
            nt.append((ct[2][0]-1, ct[2][1]+1))
            nt.append((ct[3][0], ct[3][1]))
        elif t['rotation'] == 1:
            nt.append((ct[0][0]+1, ct[0][1]-1))
            nt.append((ct[1][0], ct[1][1]-2))
            nt.append((ct[2][0]+1, ct[2][1]+1))
            nt.append((ct[3][0], ct[3][1])) 
        elif t['rotation'] == 2:
            nt.append((ct[0][0]-1, ct[0][1]-1))
            nt.append((ct[1][0]-2, ct[1][1]))
            nt.append((ct[2][0]+1, ct[2][1]-1))
            nt.append((ct[3][0], ct[3][1]))
        elif t['rotation'] == 3:
            nt.append((ct[0][0]-1, ct[0][1]+1))
            nt.append((ct[1][0], ct[1][1]+2))
            nt.append((ct[2][0]-1, ct[2][1]-1))
            nt.append((ct[3][0], ct[3][1]))
    elif t['type'] == 'Z':
        if t['rotation'] == 0:
            nt.append((ct[0][0], ct[0][1]+2))
            nt.append((ct[1][0]+1, ct[1][1]+1))
            nt.append((ct[2][0], ct[2][1]))
            nt.append((ct[3][0]+1, ct[3][1]-1))
        elif t['rotation'] == 1:
            nt.append((ct[0][0]+2, ct[0][1]))
            nt.append((ct[1][0]+1, ct[1][1]-1))
            nt.append((ct[2][0], ct[2][1]))
            nt.append((ct[3][0]-1, ct[3][1]-1)) 
        elif t['rotation'] == 2:
            nt.append((ct[0][0], ct[0][1]-2))
            nt.append((ct[1][0]-1, ct[1][1]-1))
            nt.append((ct[2][0], ct[2][1]))
            nt.append((ct[3][0]-1, ct[3][1]+1))
        elif t['rotation'] == 3:
            nt.append((ct[0][0]-2, ct[0][1]))
            nt.append((ct[1][0]-1, ct[1][1]+1))
            nt.append((ct[2][0], ct[2][1]))
            nt.append((ct[3][0]+1, ct[3][1]+1))
    elif t['type'] == 'J':
        if t['rotation'] == 0:
            nt.append((ct[0][0], ct[0][1]+2))
            nt.append((ct[1][0]-1, ct[1][1]+1))
            nt.append((ct[2][0], ct[2][1]))
            nt.append((ct[3][0]+1, ct[3][1]-1))
        elif t['rotation'] == 1:
            nt.append((ct[0][0]+2, ct[0][1]))
            nt.append((ct[1][0]+1, ct[1][1]+1))
            nt.append((ct[2][0], ct[2][1]))
            nt.append((ct[3][0]-1, ct[3][1]-1)) 
        elif t['rotation'] == 2:
            nt.append((ct[0][0], ct[0][1]-2))
            nt.append((ct[1][0]+1, ct[1][1]-1))
            nt.append((ct[2][0], ct[2][1]))
            nt.append((ct[3][0]-1, ct[3][1]+1))
        elif t['rotation'] == 3:
            nt.append((ct[0][0]-2, ct[0][1]))
            nt.append((ct[1][0]-1, ct[1][1]-1))
            nt.append((ct[2][0], ct[2][1]))
            nt.append((ct[3][0]+1, ct[3][1]+1))
    elif t['type'] == 'L':
        if t['rotation'] == 0:
            nt.append((ct[0][0]+2, ct[0][1]))
            nt.append((ct[1][0]-1, ct[1][1]+1))
            nt.append((ct[2][0], ct[2][1]))
            nt.append((ct[3][0]+1, ct[3][1]-1))
        elif t['rotation'] == 1:
            nt.append((ct[0][0], ct[0][1]-2))
            nt.append((ct[1][0]+1, ct[1][1]+1))
            nt.append((ct[2][0], ct[2][1]))
            nt.append((ct[3][0]-1, ct[3][1]-1)) 
        elif t['rotation'] == 2:
            nt.append((ct[0][0]-2, ct[0][1]))
            nt.append((ct[1][0]+1, ct[1][1]-1))
            nt.append((ct[2][0], ct[2][1]))
            nt.append((ct[3][0]-1, ct[3][1]+1))
        elif t['rotation'] == 3:
            nt.append((ct[0][0], ct[0][1]+2))
            nt.append((ct[1][0]-1, ct[1][1]-1))
            nt.append((ct[2][0], ct[2][1]))
            nt.append((ct[3][0]+1, ct[3][1]+1))
    
    blocked = False
    # make sure nothing blocks
    for i in range(0, 4):
        if df.iloc[nt[i]] == "#" and nt[i] not in t['location']:
            blocked = True
            break
        if nt[i][1] < 0 or nt[i][1] > 10:
            blocked = True
            break
    if not blocked:
        for l in t['location']:
            df.iloc[l] = ' '
        for i in range(0, 4):
            t['location'][i] = nt[i]
        for l in t['location']:
            df.iloc[l] = "#"
        t['rotation'] = (t['rotation'] + 1) % 4

In [9]:
# scan the board and clear completed rows
def clearRows():
    global df
    completed = df.loc[(df.iloc[:,0] == "#") & (df.iloc[:,1] == "#") & (df.iloc[:,2] == "#") & (df.iloc[:,3] == "#")
                 & (df.iloc[:,4] == "#") & (df.iloc[:,5] == "#") & (df.iloc[:,6] == "#") & (df.iloc[:,7] == "#")
                 & (df.iloc[:,8] == "#") & (df.iloc[:,9] == "#")]
    if completed.shape[0] == 1:
        return
    df.drop(completed.index, inplace=True)
    line = pd.DataFrame([[" "," "," "," "," "," "," "," "," "," "]], index=[0])
    for l in range(completed.shape[0]-1):
        df = pd.concat([line, df])
    line = pd.DataFrame([["#","#","#","#","#","#","#","#","#","#"]], index=[0])    
    df = pd.concat([df,line])
    resetIndex()

In [10]:
# Listen to keyboard events
keyPressed = HTML('Key Pressed')
boardEvent = Event(source=output, watched_events=['keydown'])
def handleEvent(event):
    lines = ['{}: {}'.format(k, v) for k, v in event.items()]
    content = '<br>'.join(lines)
    keyPressed.value = event['code']
    if event['code'] == 'ArrowDown':
        move("down")
    elif event['code'] == 'ArrowLeft':
        move("left")
    elif event['code'] == 'ArrowRight':
        move("right")
    elif event['code'] == 'Space':
        rotate()
    refreshBoard()
boardEvent.on_dom_event(handleEvent)

In [11]:
speed = 1.0
countDown = 100
def dropBlock():
    global countDown,speed
    move("down")
    refreshBoard()
    countDown -= 1
    if countDown == -1:
        countDown = 100
        speed *= 0.8
    timer = threading.Timer(speed, dropBlock)
    timer.start()

In [12]:
# Start game 
def startGame():
    randomTetromino()
    display(board, keyPressed)
    refreshBoard()
    dropBlock()

In [13]:
startGame()

HTML(value='Key Pressed')